In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
df = pd.read_csv('vehicle-status-prediction-data.txt')
# df = pd.read_csv('test-data.txt')

noOfcells = df['NoOfCells'][0]
print(df.shape[0])

304090


In [3]:
#variable for training
cols = list(df)[2:26]
cols

['InputVehicleId',
 'InputVehicleType',
 'InputSignalGroup',
 'InputLaneId',
 'InputApproachId',
 'InputLocationOnMap',
 'InputPhaseStatus',
 'InputSpeed',
 'InputHeading',
 'InputDistanceToStopBar',
 'InputStoppedDelay',
 'InputCellStatus',
 'OutputVehicleId',
 'OutputVehicleType',
 'OutputSignalGroup',
 'OutputLaneId',
 'OutputApproachId',
 'OutputLocationOnMap',
 'OutputPhaseStatus',
 'OutputSpeed',
 'OutputHeading',
 'OutputDistanceToStopBar',
 'OutputStoppedDelay',
 'OutputCellStatus']

In [4]:
df = df[cols].astype(float)

df.head(5)

,InputVehicleId,InputVehicleType,InputSignalGroup,InputLaneId,InputApproachId,InputLocationOnMap,InputPhaseStatus,InputSpeed,InputHeading,InputDistanceToStopBar,...,OutputSignalGroup,OutputLaneId,OutputApproachId,OutputLocationOnMap,OutputPhaseStatus,OutputSpeed,OutputHeading,OutputDistanceToStopBar,OutputStoppedDelay,OutputCellStatus
0,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0
1,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0
2,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,9.3,89.14,16.0,0.0,1.0
3,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0
4,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0


In [5]:
def getTrainingSetSize(df, noOfcells):
    trainingDataPointSize = int(df.shape[0]*0.8/noOfcells)
    training_setsize = trainingDataPointSize*noOfcells
    return training_setsize

In [6]:
# scaler = StandardScaler()
# scaler = scaler.fit(df)
# df = scaler.transform(df)
# print(df)

In [7]:
def getProcessedDataSet(df):
    input_X = []
    input_Y = []
    dataPoint_X = []
    dataPoint_Y = []
    scaler = StandardScaler()
    for index, rows in df.iterrows():
        if index == 0:
#             X_row_list = [rows.OutputVehicleId, rows.OutputVehicleType, rows.OutputSignalGroup, rows.OutputLaneId, rows.OutputApproachId, rows.OutputPhaseStatus, rows.OutputSpeed, rows.OutputHeading, rows.OutputDistanceToStopBar]
            #X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar]
            X_row_list = [rows.InputVehicleType,  rows.InputPhaseStatus, rows.InputSpeed,  rows.InputDistanceToStopBar]
                
            dataPoint_X.append(X_row_list)
            dataPoint_Y.append(rows.OutputCellStatus)
        
        elif index % noOfcells != 0:
            #X_row_list = [rows.OutputVehicleId, rows.OutputVehicleType, rows.OutputSignalGroup, rows.OutputLaneId, rows.OutputApproachId, rows.OutputPhaseStatus, rows.OutputSpeed, rows.OutputHeading, rows.OutputDistanceToStopBar]
            
            #X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar]
            X_row_list = [rows.InputVehicleType,  rows.InputPhaseStatus, rows.InputSpeed,  rows.InputDistanceToStopBar]
            dataPoint_X.append(X_row_list)            
            dataPoint_Y.append(rows.OutputCellStatus)

        elif index % noOfcells == 0 and len(dataPoint_X) > 0:
            scaler = scaler.fit(dataPoint_X)
            dataPoint_X = scaler.transform(dataPoint_X)
            
            flatList_X = [item for sublist in dataPoint_X for item in sublist]
#             flatList_Y = [item for sublist in dataPoint_Y for item in sublist]
            
            input_X.append(flatList_X)
            input_Y.append(dataPoint_Y)
            
            dataPoint_X = []
            dataPoint_Y = []
            
            #X_row_list = [rows.OutputVehicleId, rows.OutputVehicleType, rows.OutputSignalGroup, rows.OutputLaneId, rows.OutputApproachId, rows.OutputPhaseStatus, rows.OutputSpeed, rows.OutputHeading, rows.OutputDistanceToStopBar]
            #X_row_list = [rows.InputVehicleId, rows.InputVehicleType, rows.InputSignalGroup, rows.InputLaneId, rows.InputApproachId, rows.InputPhaseStatus, rows.InputSpeed, rows.InputHeading, rows.InputDistanceToStopBar]
            X_row_list = [rows.InputVehicleType,  rows.InputPhaseStatus, rows.InputSpeed,  rows.InputDistanceToStopBar]
            dataPoint_X.append(X_row_list)
            dataPoint_Y.append(rows.OutputCellStatus)
        
        if index == len(df) - 1:
            scaler = scaler.fit(dataPoint_X)
            dataPoint_X = scaler.transform(dataPoint_X)
            
            flatList_X = [item for sublist in dataPoint_X for item in sublist]
#             flatList_Y = [item for sublist in dataPoint_Y for item in sublist]
            
            input_X.append(flatList_X)
            input_Y.append(dataPoint_Y)  


#     input_X = np.array(input_X)
#     input_Y = np.array(input_Y)
    
    
    return input_X, input_Y

In [8]:
training_setsize = getTrainingSetSize(df, noOfcells)
print(training_setsize)

243272


In [9]:
df_forTraining = df.iloc[:training_setsize, 0:] # 1 means startfrom column 1

In [10]:
df_forTraining.head(5)

,InputVehicleId,InputVehicleType,InputSignalGroup,InputLaneId,InputApproachId,InputLocationOnMap,InputPhaseStatus,InputSpeed,InputHeading,InputDistanceToStopBar,...,OutputSignalGroup,OutputLaneId,OutputApproachId,OutputLocationOnMap,OutputPhaseStatus,OutputSpeed,OutputHeading,OutputDistanceToStopBar,OutputStoppedDelay,OutputCellStatus
0,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0
1,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0
2,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,9.3,89.14,16.0,0.0,1.0
3,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0
4,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.0,0.0


In [11]:
df_forTesting = df.iloc[training_setsize:, 0:]
df_forTesting.reset_index(inplace=True)

In [12]:
df_forTesting.head()

,index,InputVehicleId,InputVehicleType,InputSignalGroup,InputLaneId,InputApproachId,InputLocationOnMap,InputPhaseStatus,InputSpeed,InputHeading,...,OutputSignalGroup,OutputLaneId,OutputApproachId,OutputLocationOnMap,OutputPhaseStatus,OutputSpeed,OutputHeading,OutputDistanceToStopBar,OutputStoppedDelay,OutputCellStatus
0,243272,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,89.14,1.0,1.91,1.0
1,243273,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.00,0.0
2,243274,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.00,0.0
3,243275,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.00,0.0
4,243276,0.0,0.0,2.0,14.0,5.0,2.0,4.0,0.0,0.0,...,2.0,14.0,5.0,2.0,4.0,0.0,0.00,0.0,0.00,0.0


In [13]:
inputX,inputY = getProcessedDataSet(df_forTraining)

# print(trainX.shape)
# print(trainY.shape)

In [14]:
inputX

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [15]:
len(inputX)

5176

In [16]:
inputX = np.array(inputX)
inputY = np.array(inputY)
print(inputX.shape)
print(inputY.shape)

(5176, 188)
(5176, 47)


In [17]:
inputY

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [18]:
scaler = StandardScaler()
scaler = scaler.fit(inputY)
inputY = scaler.transform(inputY)
print(inputY.shape)

(5176, 47)


In [19]:
inputY

array([[-1.01754178, -0.89882914,  1.26634187, ..., -0.26598658,
        -0.2897914 , -0.29174527],
       [-1.01754178,  1.1125585 , -0.78967617, ..., -0.26598658,
        -0.2897914 , -0.29174527],
       [-1.01754178,  1.1125585 , -0.78967617, ..., -0.26598658,
        -0.2897914 , -0.29174527],
       ...,
       [ 0.98276063, -0.89882914, -0.78967617, ..., -0.26598658,
        -0.2897914 , -0.29174527],
       [ 0.98276063, -0.89882914, -0.78967617, ..., -0.26598658,
        -0.2897914 , -0.29174527],
       [ 0.98276063, -0.89882914, -0.78967617, ..., -0.26598658,
        -0.2897914 , -0.29174527]])

In [20]:
n_future = 1 #Number of days we want to predict into the future
n_past = 50 #Number of past days we want to use to predict the future

In [21]:
trainX = []
trainY = []

In [22]:
print(inputX.shape[0])

5176


In [23]:
for i in range (n_past, inputX.shape[0] - n_future + 1):
    

    trainX.append(inputX[i-n_past:i,])
#     trainX.append(inputX[i - n_past:i, 0:inputX[1]])

    
trainX = np.array(trainX)



In [24]:
inputY[2]

array([-1.01754178,  1.1125585 , -0.78967617, -0.72273988, -0.63759057,
       -0.54251597, -0.47962279,  2.52452359, -0.38824433, -0.3627857 ,
       -0.31934306, -0.32591299, -0.30969923, -0.2878284 , -0.3006208 ,
        3.6252002 , -0.29523984, -0.29601258, -0.27340598, -0.28861471,
       -0.29252431, -0.27584683,  3.45075799, -0.27299765, -0.28822174,
       -0.29446571, -0.27665701, -0.28307871, -0.26930249, -0.29213497,
       -0.28861471, -0.27340598, -0.28939954, -0.28939954, -0.27746549,
       -0.28625129, -0.27217965,  3.4601201 , -0.29213497, -0.27012681,
       -0.29135522, -0.28546046, -0.27176998, -0.28466809, -0.26598658,
       -0.2897914 , -0.29174527])

In [25]:
trainY=[]
# for i in range (n_past, inputY.shape[0] - n_future + 1):
    
#     print(inputY[i + n_future -1::i +n_future])

# list.append(arr[0:2:])
trainY= inputY[n_past-1:inputY.shape[0]-1:]


    
# trainY = np.array(trainY)

In [26]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (5126, 50, 188).
trainY shape == (5126, 47).


In [27]:
trainY[0]

array([ 0.98276063,  1.1125585 ,  1.26634187, -0.72273988,  1.56840462,
       -0.54251597, -0.47962279, -0.39611434, -0.38824433, -0.3627857 ,
       -0.31934306, -0.32591299, -0.30969923, -0.2878284 , -0.3006208 ,
       -0.27584683, -0.29523984, -0.29601258, -0.27340598, -0.28861471,
        3.41851934, -0.27584683, -0.2897914 ,  3.66303526, -0.28822174,
       -0.29446571, -0.27665701, -0.28307871, -0.26930249, -0.29213497,
       -0.28861471, -0.27340598, -0.28939954, -0.28939954, -0.27746549,
       -0.28625129, -0.27217965, -0.28900731, -0.29213497, -0.27012681,
       -0.29135522, -0.28546046, -0.27176998,  3.51286296, -0.26598658,
       -0.2897914 , -0.29174527])

In [28]:
model = Sequential()
model.add(LSTM(64, activation = 'relu', input_shape =(trainX.shape[1],trainX.shape[2]), return_sequences = True))
model.add(LSTM(32, activation = 'relu', return_sequences = False))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(trainY.shape[1]))

model.compile(optimizer = 'adam', loss = 'mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 64)            64768     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 47)                1551      
Total params: 78,735
Trainable params: 78,735
Non-trainable params: 0
_________________________________________________________________


In [29]:
history = model.fit(trainX, trainY, epochs =10, batch_size = 16, validation_split= 0.1, verbose =1)



Train on 4613 samples, validate on 513 samples
Epoch 1/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9670 - val_loss: 1.0355
Epoch 2/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9499 - val_loss: 1.0246
Epoch 3/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9461 - val_loss: 1.0219
Epoch 4/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9418 - val_loss: 1.0066
Epoch 5/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9394 - val_loss: 1.0125
Epoch 6/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9366 - val_loss: 1.0031
Epoch 7/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9356 - val_loss: 1.0191
Epoch 8/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9333 - val_loss: 1.0102
Epoch 9/10
4613/4613 [==============================] - 9s 2ms/step - loss: 0.9318 - val_loss: 1.0130
Epoch 10/10
4613/4613 [============

In [30]:
test_inputX, test_inputY = getProcessedDataSet(df_forTesting)

In [31]:
test_inputX = np.array(test_inputX)
test_inputY = np.array(test_inputY)
print(test_inputX.shape)
print(test_inputY.shape)

(1294, 188)
(1294, 47)


In [32]:
scaler = StandardScaler()
scaler = scaler.fit(test_inputY)
test_inputY = scaler.transform(test_inputY)

In [33]:
testX = []
for i in range (n_past, test_inputX.shape[0] - n_future + 1):
    
    testX.append(test_inputX[i-n_past:i,])
    
testX = np.array(testX)


In [34]:
testY=[]
testY= test_inputY[n_past-1:test_inputY.shape[0]-1:]

In [35]:
print('testX shape == {}.'.format(testX.shape))
print('testY shape == {}.'.format(testY.shape))
print('trainY shape == {}.'.format(trainY.shape))

testX shape == (1244, 50, 188).
testY shape == (1244, 47).
trainY shape == (5126, 47).


In [36]:
# make predictions

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
print('trainY shape == {}.'.format(trainY.shape))

trainY shape == (5126, 47).


In [37]:
#trainPredict
# invert predictions
# scaler = StandardScaler()
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY)

In [38]:
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY)
print('testY shape == {}.'.format(testY.shape))

testY shape == (1244, 47).


In [39]:
trainPredict

array([[0.56953204, 0.44551572, 0.38246977, ..., 0.06540243, 0.06837631,
        0.07746127],
       [0.56953204, 0.44551572, 0.38246977, ..., 0.06540243, 0.06837631,
        0.07746127],
       [0.56953204, 0.44551572, 0.38246977, ..., 0.06540243, 0.06837631,
        0.07746127],
       ...,
       [0.56953204, 0.44551572, 0.38246977, ..., 0.06540243, 0.06837631,
        0.07746127],
       [0.56953204, 0.44551572, 0.38246977, ..., 0.06540243, 0.06837631,
        0.07746127],
       [0.56953204, 0.44551572, 0.38246977, ..., 0.06540243, 0.06837631,
        0.07746127]], dtype=float32)

In [40]:
trainY.shape

(5126, 47)

In [41]:
trainPredict.shape

(5126, 47)

In [42]:
trainY[:,].shape

(5126, 47)

In [43]:
import math
from sklearn.metrics import mean_squared_error
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))

In [44]:
trainScore

0.442775123091363

In [45]:
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))

In [46]:
testScore

0.4542344385626543